In [ ]:
import pandas as pd
import numpy as np
import os

pd.set_option("display.max_rows", None)

pathAlquiler = "../../data/raw/alquiler/"
pathVenta = "../../data/raw/venta/"
pathDataProcessed = "../../data/interim/"

## Notas metodológicas

- Los precios de venta y alquiler se han obtenido a partir de datos de **oferta inmobiliaria**, no de transacciones cerradas.
- El **precio medio anual** se ha calculado como la **media aritmética de los precios mensuales** disponibles para cada comunidad autónoma y año.
- En aquellos casos en los que no existen datos de venta o alquiler para una determinada comunidad autónoma y año, el valor se representa como **NaN**.
- Los precios están expresados en **euros corrientes**.


In [2]:
contenido_venta = os.listdir(pathVenta)
contenido_alquiler = os.listdir(pathAlquiler)
print(contenido_venta)
print(contenido_alquiler)

['Andalucia.csv', 'Aragon.csv', 'Asturias.csv', 'Baleares.csv', 'Canarias.csv', 'Cantabria.csv', 'Castilla y León.csv', 'Castilla-La Mancha.csv', 'Cataluña.csv', 'Ceuta.csv', 'Comunidad Valenciana.csv', 'Euskadi.csv', 'Extremadura.csv', 'Galicia.csv', 'La Rioja.csv', 'Madrid.csv', 'Melilla.csv', 'Murcia.csv', 'Navarra.csv']
['Andalucia.csv', 'Aragon.csv', 'Asturias.csv', 'Baleares.csv', 'Canarias.csv', 'Cantabria.csv', 'Castilla y León.csv', 'Castilla-La Mancha.csv', 'Cataluña.csv', 'Ceuta.csv', 'Comunidad Valenciana.csv', 'Euskadi.csv', 'Extremadura.csv', 'Galicia.csv', 'La Rioja.csv', 'Madrid.csv', 'Melilla.csv', 'Murcia.csv', 'Navarra.csv']


In [3]:
lista_df_venta = []
lista_df_alquiler = []

for ccaa in contenido_venta:
    lista_df_venta.append(pd.read_csv(pathVenta + ccaa))

for ccaa in contenido_alquiler:
    lista_df_alquiler.append(pd.read_csv(pathAlquiler + ccaa))

In [ ]:
df_total_venta = pd.concat(
    lista_df_venta,
    axis=0,           # apilar filas
    ignore_index=True # reindexar
)


df_total_alquiler = pd.concat(
    lista_df_alquiler,
    axis=0,           # apilar filas
    ignore_index=True # reindexar
)

df_total_venta = df_total_venta.drop(["Unnamed: 0", "Variación mensual", "Variación trimestral", "Variación anual"], axis = 1)
df_total_venta = df_total_venta.replace("n.d.", np.nan)
df_total_venta = df_total_venta.dropna()

df_total_venta["Precio m2"] = (
    df_total_venta["Precio m2"]
    .str.replace("€/m2", "", regex=False)
    .str.replace(".", "", regex=False)   # elimina miles
    .str.replace(",", ".", regex=False)  # decimal
    .astype(float)
)


df_total_venta["Año"] = df_total_venta["Mes"].str.extract(r"(\d{4})").astype(int)

df_total_venta = (
    df_total_venta
    .groupby(["Comunidad Autonoma", "Año"], as_index=False)
    .agg(precio_medio_anual_eur_m2_venta=("Precio m2", "mean"))
)

df_total_venta["precio_medio_anual_eur_m2_venta"] = df_total_venta["precio_medio_anual_eur_m2_venta"].round(2)

# display(df_total_venta)

df_total_alquiler = df_total_alquiler.drop(["Unnamed: 0", "Variación mensual", "Variación trimestral", "Variación anual"], axis = 1)
df_total_alquiler = df_total_alquiler.replace("n.d.", np.nan)
df_total_alquiler = df_total_alquiler.dropna()

df_total_alquiler["Precio m2"] = (
    df_total_alquiler["Precio m2"]
    .str.replace("€/m2", "", regex=False)
    .str.replace(".", "", regex=False)   # elimina miles
    .str.replace(",", ".", regex=False)  # decimal
    .astype(float)
)

df_total_alquiler["Año"] = df_total_alquiler["Mes"].str.extract(r"(\d{4})").astype(int)

df_total_alquiler = (
    df_total_alquiler
    .groupby(["Comunidad Autonoma", "Año"], as_index=False)
    .agg(precio_medio_anual_eur_m2_alquiler=("Precio m2", "mean"))
)

df_total_alquiler["precio_medio_anual_eur_m2_alquiler"] = df_total_alquiler["precio_medio_anual_eur_m2_alquiler"].round(2)

# display(df_total_alquiler)

df_final = df_total_venta.merge(
    df_total_alquiler,
    on=["Comunidad Autonoma", "Año"],
    how="outer"
)

df_final = df_final.dropna()
display(df_final)
df_final.to_csv(pathDataProcessed + "alquiler_venta_ccaa.csv")

,Comunidad Autonoma,Año,precio_medio_anual_eur_m2_venta,precio_medio_anual_eur_m2_alquiler
0,Andalucia,2006,1967.75,9.60
1,Andalucia,2007,1915.33,8.62
2,Andalucia,2008,1839.67,7.99
3,Andalucia,2009,1757.50,7.27
4,Andalucia,2010,1713.33,6.85
5,Andalucia,2011,1723.42,6.72
6,Andalucia,2012,1579.42,6.43
7,Andalucia,2013,1443.42,6.04
8,Andalucia,2014,1366.00,5.95
9,Andalucia,2015,1372.33,6.12


## Construcción del dataset final de precios de venta y alquiler por CCAA

En esta parte del proyecto se realiza la **integración, limpieza y transformación** de los datos inmobiliarios de **venta y alquiler** para todas las **Comunidades Autónomas**, con el objetivo de obtener un único conjunto de datos anual y comparable.

---

### Carga de datos por Comunidad Autónoma

Los datos originales se encuentran almacenados en archivos CSV separados por:
- Tipo de operación (`Venta` y `Alquiler`)
- Comunidad Autónoma

Primero se listan los archivos disponibles en cada carpeta y se cargan todos los CSV en memoria, generando:
- Una lista de `DataFrames` de **venta**
- Una lista de `DataFrames` de **alquiler**

Posteriormente, cada lista se concatena verticalmente para obtener:
- Un único `DataFrame` con **todos los precios de venta**
- Un único `DataFrame` con **todos los precios de alquiler**

---

### Limpieza y preprocesado de los datos

Sobre ambos conjuntos de datos se aplican los mismos pasos de limpieza:

- Eliminación de columnas que no son relevantes para el análisis:
  - Índices automáticos
  - Variaciones porcentuales (mensual, trimestral y anual)

  Estas variaciones se descartan porque el análisis se centra en el **impacto directo sobre el precio**, y no en los porcentajes proporcionados originalmente.

- Sustitución de valores `"n.d."` por `NaN` y eliminación de filas incompletas.
- Limpieza y conversión de la columna **`Precio m2`**:
  - Eliminación del símbolo `€/m2`
  - Normalización del formato numérico
  - Conversión a tipo `float`

---

### Agregación temporal

A partir de la columna **`Mes`**, se extrae el **año** y se realiza una agregación anual:

- Se calcula el **precio medio anual por metro cuadrado**
- Se agrupa por:
  - Comunidad Autónoma
  - Año

El resultado son dos `DataFrames`:
- Precio medio anual de **venta** por CCAA
- Precio medio anual de **alquiler** por CCAA

Ambos valores se redondean a dos decimales para facilitar su interpretación.

---

### Integración final de venta y alquiler

Finalmente, los datos de venta y alquiler se combinan en un único `DataFrame` mediante un `merge` usando como claves:
- `Comunidad Autonoma`
- `Año`

Se eliminan las filas con valores nulos resultantes del cruce, ya que no es posible imputar precios inexistentes sin introducir sesgos en el análisis.

---

### Resultado final

El dataset final contiene, para cada **Comunidad Autónoma y año**:

- Precio medio anual de venta por m²
- Precio medio anual de alquiler por m²



## Diccionario de datos

A continuación se describe la estructura y el significado de las variables incluidas en el conjunto de datos utilizado para el análisis exploratorio (EDA) de precios de vivienda en España.

| Columna                             | Descripción                                                                 | Tipo de Variable                              | Importancia (0–3) | Notas |
|-------------------------------------|-----------------------------------------------------------------------------|-----------------------------------------------|------------------|-------|
| **Comunidad Autonoma**              | Comunidad autónoma de España a la que pertenecen los precios                | Categórica nominal (clave geográfica)         | 3                | 19 comunidades autónomas más Ceuta y Melilla |
| **Año**                             | Año natural al que corresponde el precio medio                              | Temporal discreta (entero)                     | 3                | Formato YYYY; extraído de datos mensuales |
| **precio_medio_anual_eur_m2_venta** | Precio medio anual de oferta de vivienda en venta por metro cuadrado (€)   | Numérica continua (float)                      | 3                | Media aritmética de precios mensuales; puede contener valores NaN |
| **precio_medio_anual_eur_m2_alquiler** | Precio medio anual de oferta de vivienda en alquiler por metro cuadrado (€) | Numérica continua (float)                      | 3                | Media aritmética de precios mensuales; puede contener valores NaN |

